In [1]:
# Global inversion with empymod

#%matplotlib notebook

In [2]:
import empymod
import numpy as np
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed

In [3]:

# Receivers geometry

offsets = np.array([2, 4, 8]) # in meters
dip = np.array([0, 90])

Hreceivers = [offsets, offsets*0, 0, 0, 0]
Vreceivers = [offsets, offsets*0, 0, 0, 90]
Preceivers = [offsets, offsets*0, 0, 90, 90]

# Source geometry

Hsource = [0, 0, 0 ,0 , 0]
Vsource = [0, 0, 0, 0, 90]

# Frequency

freq = 9000

In [4]:
# True data

res = [2e14, 10, 20, 10]
depth = [0, 2, 5]

HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop')
PRP = empymod.loop(Hsource, Preceivers, depth, res, freq, xdirect=None, mrec = 'loop')

Zdata = np.hstack((HCP, VCP, PRP))



:: empymod END; runtime = 0:00:00.539414 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.003918 :: 1 kernel call(s)


:: empymod END; runtime = 0:00:00.004369 :: 1 kernel call(s)



In [9]:
# sampling of depth and conductivities
nsl = 21

s0 = -2 # minimum conductivity in S/m
s1 = -0.8 # maximum conductivity in S/m
# conductivities array
conds = np.logspace(s0, s1, nsl)

th0 = 0.1 # minimum thickness in m
th1 = 5   # maximum thickness in m
# thickness array
thicks = np.linspace(th0, th1, nsl)

# Array to store values

#Zcube = np.zeros((nsl, nsl, nsl, nsl, nsl, 9), dtype = 'complex') # 9 coil geometries, 5 parameters 

In [10]:
# Loop to create hypercube

def forward_parallel(is1, is2, is3, it1, it2):
    time.sleep(0.01)
    
    res[1] = 1/conds[is1] # set resistivity of first layer
    res[2] = 1/conds[is2] # set resistivity of second layer
    res[3] = 1/conds[is3] # set resistivity of third layer
    depth[1] = thicks[it1] # set thickness of first layer
    depth[2] = depth[1] + thicks[it2] # set thickness of second layer

    # Compute fields

    HCP = empymod.loop(Hsource, Hreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    VCP = empymod.loop(Vsource, Vreceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)
    PRP = empymod.loop(Hsource, Preceivers, depth, res, freq, xdirect=None, mrec = 'loop', verb = 0)

    # Store in hypercube

    #Zcube[is1, is2, is3, it1, it2, 0:3] = HCP
    #Zcube[is1, is2, is3, it1, it2, 3:6] = VCP
    #Zcube[is1, is2, is3, it1, it2, 6:9] = PRP

    Z = np.hstack((HCP, VCP, PRP))
    return Z

In [11]:
startTime = time.time()

Results = Parallel(n_jobs=-1,verbose=1)(delayed(forward_parallel)(i, j, k, m, n) for i in range(0,nsl) 
        for j in range(0,nsl) for k in range(0,nsl) for m in range(0,nsl) for n in range (0,nsl))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2784 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 5584 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 9184 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 13584 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 18784 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 24784 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 31584 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 39184 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 47584 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 56784 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 66784 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 77584 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 89184 t

Execution time in seconds: 5861.9494025707245


In [12]:
np.save('Results', Results)

In [ ]:
ns_array = np.array([5, 10, 15, 51])

time_8work = np.array([10, 278, 2075, 942785]) # guessing the last
time_loop = np.array([19, 649, 265.6*15, 2097753]) # guessing the last
time_48work = np.array([3.6, 32, 221.14, 30*60*60]) # guessing the last two


plt.semilogy(ns_array, time_loop, 'k', label = 'Loop')
plt.semilogy(ns_array, time_8work, 'b', label = '8 workers')
plt.semilogy(ns_array, time_48work, 'r', label = '48 workers')
plt.xlabel('Number of samples')
plt.ylabel('Seconds')
plt.legend()
